In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CelebA
import torchvision.models as models

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

celeba_dataset = CelebA(root='./data', split='all', transform=transform, download=True)

lfw_people = LFW(root='./data', download=True, transform=transform)
dataset_size = len(lfw_people)
train_size = int(0.75 * dataset_size)
test_size = dataset_size - train_size
train_dataset, test_dataset = random_split(lfw_people, [train_size, test_size])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

transform = transforms.Compose([transforms.ToPILImage(),  # Convert to PIL Image
                                transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
                                transforms.Resize((h, w)),  # Resize to the original dimensions
                                transforms.ToTensor()])

X_train = torch.stack([transform(img.reshape(h, w)) for img in X_train])
X_test = torch.stack([transform(img.reshape(h, w)) for img in X_test])

class CNNClassifier(nn.Module):
    def _init_(self, num_classes):
        super(CNNClassifier, self)._init_()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3)
        self.fc1 = nn.Linear(128 * (h - 4) * (w - 4), 256)  # Adjust the input size based on your dataset dimensions
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

batch_size = 64
train_dataset = torch.utils.data.TensorDataset(X_train, torch.tensor(y_train, dtype=torch.int64))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torch.utils.data.TensorDataset(X_test, torch.tensor(y_test, dtype=torch.int64))
test_loader = DataLoader(test_dataset, batch_size=batch_size)

num_classes = len(lfw_people.target_names)
model = CNNClassifier(num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 20

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 0):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics every few batches
        running_loss += loss.item()
        if i % 10 == 9:  # Print every 10 batches
            print(f"Epoch [{epoch+1}/{epochs}], Batch [{i+1}/{len(train_loader)}], Loss: {running_loss/10:.4f}")
            running_loss = 0.0

print("Training finished.")

model.eval()
correct = 0
total = 0
all_labels = []
all_predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

accuracy = 100 * correct / total
print(f"Accuracy: {accuracy:.2f}%")
print(classification_report(all_labels, all_predictions, target_names=lfw_people.target_names))